In [9]:
import pandas as pd
from shapely import geometry
import mercantile
from tqdm import tqdm
import os
import tempfile

In [3]:
import geopandas as gpd
import pygris
from pygris.data import get_census

## 1 get building footprint

In [ ]:
# the adopted code is from https://github.com/microsoft/GlobalMLBuildingFootprints.git. Credits to the contributors and Microsoft.

# get bbox from https://geojson.io
# this method will simplified in the future
aoi_geom = {
    "coordinates": [
        [
            [
                -83.0702275396326,
                42.32975744601228
            ],
            [
                -83.0702275396326,
                42.325914309498785
            ],
            [
                -83.06144254245925,
                42.325914309498785
            ],
            [
                -83.06144254245925,
                42.32975744601228
            ],
            [
                -83.0702275396326,
                42.32975744601228
            ]
        ]
    ],
    "type": "Polygon"
}
aoi_shape = geometry.shape(aoi_geom)
# get bbox definition
minx, miny, maxx, maxy = aoi_shape.bounds

output_fn = "building_footprints.geojson"

# get tiles intersect bbox
quad_keys = set()
for tile in list(mercantile.tiles(minx, miny, maxx, maxy, zooms=9)):
    quad_keys.add(mercantile.quadkey(tile))
quad_keys = list(quad_keys)
print(f"The input area spans {len(quad_keys)} tiles: {quad_keys}")

# Download the building footprints for each tile and crop with bbox
df = pd.read_csv(
    "https://minedbuildings.z5.web.core.windows.net/global-buildings/dataset-links.csv", dtype=str
)

idx = 0
combined_gdf = gpd.GeoDataFrame()
with tempfile.TemporaryDirectory() as tmpdir:
    # Download the GeoJSON files for each tile that intersects the input geometry
    tmp_fns = []
    for quad_key in tqdm(quad_keys):
        rows = df[df["QuadKey"] == quad_key]
        if rows.shape[0] == 1:
            url = rows.iloc[0]["Url"]

            df2 = pd.read_json(url, lines=True)
            df2["geometry"] = df2["geometry"].apply(geometry.shape)

            gdf = gpd.GeoDataFrame(df2, crs=4326)
            fn = os.path.join(tmpdir, f"{quad_key}.geojson")
            tmp_fns.append(fn)
            if not os.path.exists(fn):
                gdf.to_file(fn, driver="GeoJSON")
        elif rows.shape[0] > 1:
            raise ValueError(f"Multiple rows found for QuadKey: {quad_key}")
        else:
            raise ValueError(f"QuadKey not found in dataset: {quad_key}")

    # Merge the GeoJSON files into a single file
    for fn in tmp_fns:
        gdf = gpd.read_file(fn)  # Read each file into a GeoDataFrame
        gdf = gdf[gdf.geometry.within(aoi_shape)]  # Filter geometries within the AOI
        gdf['id'] = range(idx, idx + len(gdf))  # Update 'id' based on idx
        idx += len(gdf)
        combined_gdf = pd.concat([combined_gdf,gdf],ignore_index=True)

## 2 get block and parcel data (census data)

In [ ]:
# get the block polygon data
mi_blocks = pygris.blocks(state = "MI", year=2020, county = "Wayne")

Using FIPS code '26' for input 'MI'
Using FIPS code '163' for input 'Wayne'


In [5]:
mi_blocks

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,HOUSING20,POP20,geometry
46,26,163,574600,3026,261635746003026,Block 3026,G5040,U,23824,U,S,11227,0,+42.3135927,-083.2651531,13,35,"POLYGON ((-83.26578 42.31408, -83.26456 42.314..."
47,26,163,574600,1027,261635746001027,Block 1027,G5040,U,23824,U,S,11743,0,+42.3135895,-083.2639106,13,39,"POLYGON ((-83.26456 42.31411, -83.26334 42.314..."
48,26,163,522300,1040,261635223001040,Block 1040,G5040,U,23824,U,S,6761,0,+42.3527259,-083.0934920,2,9,"POLYGON ((-83.09428 42.35276, -83.09297 42.353..."
49,26,163,536500,1010,261635365001010,Block 1010,G5040,U,23824,U,S,2447,0,+42.3893940,-083.1424037,0,0,"POLYGON ((-83.14299 42.38929, -83.14188 42.389..."
50,26,163,586201,4018,261635862014018,Block 4018,G5040,U,23824,U,S,63681,0,+42.1822076,-083.3143189,10,31,"POLYGON ((-83.31591 42.18329, -83.31534 42.183..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254386,26,163,524500,1000,261635245001000,Block 1000,G5040,U,23824,U,S,0,12472,+42.2925470,-083.1439701,0,0,"POLYGON ((-83.14534 42.29324, -83.14453 42.293..."
254387,26,163,500200,2031,261635002002031,Block 2031,G5040,U,23824,U,S,22458,0,+42.4433329,-082.9572393,18,38,"POLYGON ((-82.95848 42.44381, -82.95604 42.443..."
254405,26,163,585800,1002,261635858001002,Block 1002,G5040,U,23824,U,S,0,5415,+42.2607878,-083.4161392,0,0,"POLYGON ((-83.41629 42.2619, -83.4161 42.26233..."
254407,26,163,982302,1001,261639823021001,Block 1001,G5040,U,23824,U,S,0,40441,+42.1315611,-083.1837561,0,0,"POLYGON ((-83.18406 42.12911, -83.18406 42.129..."


In [ ]:
# get the census data for the block
# need to chack the variables for the census data in r using tidycensus???
mi_tract = get_census(dataset = "acs/acs5/profile",
                        variables = "DP02_0068PE",
                        year = 2020,
                        params = {
                          "for": "tract:*",
                          "in": "state:26 county:163"},
                        guess_dtypes = True,
                        return_geoid = True)

In [7]:
mi_tract

,DP02_0068PE,GEOID
0,70.2,26163550900
1,67.2,26163551100
2,60.7,26163551200
3,27.9,26163551300
4,31.6,26163551400
...,...,...
622,67.4,26163550400
623,71.3,26163550500
624,81.3,26163550600
625,75.8,26163550700


In [ ]:
# join the block and census data

## 3 calcualte metrics on different level